In [15]:
# Testing pipeline for the QSM 3D segmentation project 
# Course: cs512
# instructor: Gady Agam
# Group 1: 
# Rasheed Abid, rabid@hawk.iit.edu
# Khalid Saifullah, ksaifullah@hawk.iit.edu


In [16]:
# installations if required
!pip install patchify

In [17]:
# imports
import os
import numpy as np
import nibabel as nib
from matplotlib import pyplot as plt

from skimage import io
from sklearn.model_selection import train_test_split

from patchify import patchify, unpatchify

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
import keras
from keras import backend as K
from keras.models import Model
from keras.models import load_model
from keras.layers import Input, Conv3D, MaxPooling3D, UpSampling3D, concatenate, Conv3DTranspose, BatchNormalization, Dropout, Lambda
from keras.optimizers import Adam
from keras.layers import Activation, MaxPool2D, Concatenate

print("Tensorflow version = " + tf.__version__)
print("Keras version = " + keras.__version__)


Tensorflow version = 2.11.1
Keras version = 2.11.0


In [18]:
# # incase of running on google colab
# from google.colab import drive
# drive.mount('/content/drive')

In [19]:
# changing variables

# maindir = "/content/drive/MyDrive/Colab Notebooks/CS512/project"
maindir = "/Users/rashid_abid/Desktop/CS512_Computer_Vision/Project_proposal/cs512_project_gdrive_runs"
subject_num = 80 # available ones are 32, 64, 80, 100
subject_num_testing = 9 # available ones are 9, 20
label_name = "hippocampus" # available ones are "caudate", "putamen", "hippocampus", "thalamus", "amygdala"
epochs = 50
dim = 128 # available ones are 64, 128, 256
n_classes = 1

# architecture parameters
patch_size = 64 # testing with 64x64x64 patches
channels=1

LR = 0.0001 # Learning rate
optim = keras.optimizers.Adam(LR)

In [20]:
# directories 

os.chdir( maindir )
source_dir = os.path.join(maindir, "data_dir")
results_dir = os.path.join(maindir, "results_dir")
files_dir = os.path.join(maindir, "files")

In [21]:
# file names
saved_model_name = label_name + "_mask_dim" + str(dim) + "_sub" + str(subject_num) + "_e" + str(epochs) + ".h5"
saved_model_name = os.path.join(results_dir, "saved_models", saved_model_name)

subject_filename = "filename_sub" + str(subject_num) + ".txt"
filename = os.path.join(files_dir, subject_filename)
test_filename = os.path.join(files_dir, "filename_testing" + str(subject_num_testing) + ".txt")


print(saved_model_name)

/Users/rashid_abid/Desktop/CS512_Computer_Vision/Project_proposal/cs512_project_gdrive_runs/results_dir/saved_models/hippocampus_mask_dim128_sub80_e50.h5


In [22]:
# suffix selection for images
if dim == 256:
    Suffix = 2
elif dim ==  128:
    Suffix = 4
else:
    Suffix = 8

print( "Suffix for the files is = %d" %Suffix )

Suffix for the files is = 4


In [23]:
# # Make sure the GPU is available. 
# import tensorflow as tf
# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#   raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))

In [24]:
# function to normalize the nifti data once converted to numpy array, and then multiply by the mask
# of the brain

def norm_nifti(img, brain_mask_data):
    # We are normalizing the data to be between 0 and 1
    min_value = np.min(img)
    max_value = np.max(img)
    normalized_data = (img - min_value) / (max_value - min_value)
    
    # We are multiplying the normalized data by the brain mask
    # to make sure that the background is 0
    normalized_data = normalized_data * brain_mask_data
    
    return normalized_data

In [25]:
# Loss functions and dice metrics
# both for binary and multiclass segmentation
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    smoothing_factor = 1 # to avoid division by zero
    
    intersection = K.sum(y_true_f * y_pred_f)
    union = K.sum(y_true_f) + K.sum(y_pred_f)
    dice = (2. * intersection + smoothing_factor) / (union + smoothing_factor)
    return dice

def dice_coef_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)

def dice_coef_multilabel(y_true, y_pred, numLabels=1):
    dice=0
    for index in range(numLabels):
        dice -= dice_coef(y_true[...,index], y_pred[...,index])
    return dice/numLabels


In [26]:
# print the name of the model to check
print("Loading the model from the saved file: " + saved_model_name)

# load the model
model = load_model(saved_model_name, custom_objects={'dice_coef_loss': dice_coef_loss, 'dice_coef': dice_coef})                  
# model = keras.models.load_model(saved_model_name) # this one does not work for custom loss functions

Loading the model from the saved file: /Users/rashid_abid/Desktop/CS512_Computer_Vision/Project_proposal/cs512_project_gdrive_runs/results_dir/saved_models/hippocampus_mask_dim128_sub80_e50.h5


In [27]:
# Testing phase with unseen dataset

In [28]:
test_filename = test_filename
reconstructed_dir = os.path.join( results_dir, "reconstructed_predictions" )
# check if the directory exists, if not create it
if not os.path.exists(reconstructed_dir):
    os.makedirs(reconstructed_dir)

all_img_patches = []
all_mask_patches = []
all_dice_scores = []
val=0

print("The test dataset is being processed now....")
with open(test_filename) as f:
    for line in f:
        print("Currently processing subject number: %d" %val)
        val=val+1
        subject_id = line.strip()

        # Read file names
        imagename =  os.path.join(source_dir, subject_id, "QSM_masked_dim" + str(dim) + ".nii.gz") 
        maskname = os.path.join(source_dir, subject_id, label_name + "_dim" + str(dim) + ".nii.gz") 
        brain_mask_name = os.path.join(source_dir, subject_id, "QSM_brain_mask_dim" + str(dim) + ".nii.gz") 

        # Read images
        img = nib.load(imagename)
        img_data = img.get_fdata().astype(np.float32)
        
        # Read mask
        mask = nib.load(maskname)
        mask_data = mask.get_fdata().astype(np.float32)

        # Read brain mask
        brain_mask = nib.load(brain_mask_name)
        brain_mask_data = brain_mask.get_fdata().astype(np.float32)
        
        #Normalise using the function we wrote
        img_data = norm_nifti(img_data, brain_mask_data)
        
        #Patchify the image and mask
        patches_img = patchify(img_data, (patch_size, patch_size, patch_size), step=64)  # Step=256 for 256 patches means no overlap
        patches_mask = patchify(mask_data, (patch_size, patch_size, patch_size), step=64)  # Step=256 for 256 patches means no overlap
        

        # get a temporary block to store the predicted patches for the current subject
        temporary_blocks = np.empty(patches_img.shape)

        dice_scores = []
        for i in range(patches_img.shape[0]):
            for j in range(patches_img.shape[1]):
                for k in range(patches_img.shape[2]):
                    
                    #image
                    single_patch_img = patches_img[i,j,k,:,:,:]
                    all_img_patches.append(single_patch_img)
                    
                    #mask
                    single_patch_mask = patches_mask[i,j,k,:,:,:]
                    all_mask_patches.append(single_patch_mask)

                    ### predicting on the patches
                    temp_img_patch = np.expand_dims(single_patch_img, -1) # to fit the model input
                    temp_img_patch = np.expand_dims(temp_img_patch, 0) # to fit the model input
                    current_predict = model.predict(temp_img_patch, verbose=0) # predict
                    current_predict = (current_predict>0.5).astype(np.float32)
                    # get the dice score and save it in a list
                    dice_score = dice_coef(single_patch_mask, current_predict)
                    dice_scores.append(dice_score)

                    temporary_blocks[i,j,k,...] = np.squeeze(current_predict) # save the prediction in the temporary block

        
        # print the average dice score for the current subject
        print("The average dice score for the current subject is: " + str(np.mean(dice_scores)))
        # save the mean dice scores for all subjects in a list
        all_dice_scores.append(np.mean(dice_scores))

        # save the predicted patches for the current subject in the reconstructed images in the results folder
        reconstructed_image = unpatchify(temporary_blocks, img_data.shape)
        reconstructed_imges = nib.Nifti1Image(reconstructed_image, img.affine, img.header)

        # check first, and then make a directory for the subject in the reconstructed folder, if it does not exist
        if not os.path.exists(os.path.join(reconstructed_dir, str(subject_id))):
            os.mkdir(os.path.join(reconstructed_dir, str(subject_id)))

        recostructed_filename = os.path.join(reconstructed_dir, str(subject_id), label_name + "_dim" + str(dim) + "_e" + str(epochs) +  "_recon_by_" + str(Suffix) + ".nii.gz")
        nib.save(reconstructed_imges, recostructed_filename)
        ## reconstruction is done for the current subject
        

        # image stacking 
        all_patched_stacked_images = np.array(all_img_patches)
        # mask stacking
        all_patched_stacked_masks = np.array(all_mask_patches)



The test dataset is being processed now....
Currently processing subject number: 0
The average dice score for the current subject is: 0.7193836
Currently processing subject number: 1
The average dice score for the current subject is: 0.8493917
Currently processing subject number: 2
The average dice score for the current subject is: 0.85301983
Currently processing subject number: 3
The average dice score for the current subject is: 0.835168
Currently processing subject number: 4
The average dice score for the current subject is: 0.604786
Currently processing subject number: 5
The average dice score for the current subject is: 0.8254431
Currently processing subject number: 6
The average dice score for the current subject is: 0.81360793
Currently processing subject number: 7
The average dice score for the current subject is: 0.87856996
Currently processing subject number: 8
The average dice score for the current subject is: 0.7315997


In [29]:
# print the average dice score for all subjects
print("The average dice score for all " + str(subject_num_testing) +  " subjects is: " + str(np.mean(all_dice_scores)))

The average dice score for all 9 subjects is: 0.7901078
